In [1]:
from simple_rag.chat.chat_openai import ChatOpenAI
from simple_rag.utils import connect_to_database, insert_to_db

## Get Connection to Database

In [2]:
collection = connect_to_database()

🔥 Successfully Get Access Database


## Initialize Large Language Model Object

In [3]:
llm = ChatOpenAI()

## Case Study 1: System's prompt take a short instruct from user's prompt

In [4]:
system_prompt = """
    You possess in-depth knowledge of natural images and can describe them with ease. \
    From the given input text indicating the category name of a certain object, your task involves the following steps:
    1-Imagine a scene containing the input object.
    2-Generate 4 descriptions about different key appearance features of the input object from the imagined scene, with each description having a maximum of 16 words.
    3-Output a JSON object containing the following key:
        "description": <list of 4 descriptions>

    Use the following examples:
        Input text: "sea lion"
        Answer: "description": ["A round-bellied seal sits on a rock, looking intently at something off-camera.", "The seal lies with flippers tucked, sleek body well-maintained.", "The seal's thick, smooth fur and large dark eyes show alertness and curiosity.", "Turquoise water contrasts with the seal's brown fur and grey rock, highlighting its natural environment."]
"""
response = llm.request(system_prompt=system_prompt, user_prompt="British shorthair")
insert_to_db(collection=collection, data=response)
print(response)

🔥 Successfully Log Request to Database


{'request_id': 'chatcmpl-9sPcmQGIQ0wcBI1NS3Zfz8QoeDYOI',
 'output': {'description': ['A fluffy British Shorthair cat lounges on a cozy window sill, gazing outside.',
   'Its round face and large, expressive eyes give off a sweet and gentle expression.',
   "The cat's dense, plush coat in a solid color exudes a luxurious and elegant look.",
   "Sunlight filters through the window, casting a warm glow on the cat's adorable features."]},
 'completion_tokens': 82,
 'prompt_tokens': 217,
 'total_tokens': 299}

## Case Study 2: System's prompt take a long instruct from user's prompt

In [5]:
structured_sample = {
    "id": "user_oriented_task_0",
    "motivation_app": "Grammarly",
    "instruction": "The sentence you are given might be too wordy, complicated, or unclear. Rewrite the sentence and make your writing clearer by keeping it concise. Whenever possible, break complex sentences into multiple sentences and eliminate unnecessary words.",
    "instances": [
        {
            "input": "If you have any questions about my rate or if you find it necessary to increase or decrease the scope for this project, please let me know.",
            "output": "If you have any questions about my rate or find it necessary to increase or decrease this project's scope, please let me know.",
        }
    ],
}

def gen_prompt_no_input(ins, outp):

    sys_prompt = "You are a helpful, precise but picky assistant for checking the quality of the answer to a given instruction."
    prompt_template = "[Instruction]\n{ins}\n\n[The Start of Answer]\n{outp}\n\n[The End of Answer]\n\n[System]\n{criteria}\n\n"
    criteria = (
        "We would like you to answer several questions related to the quality of the answer to the given instruction. \n"
        + "1. Why this answer is not good for the given instruction? Analyse based on the Helpfulness, Relevance, Accuracy, Level of Details, and Structure. \n"
        + "2. Based on the reason you provided, please generate a better answer while preserving the same content. To achieve that, you may want to adjust the level of details, add bullet points, or use comprehensive words, etc. The answer should be in the format of [Better Answer] your answer [End] \n"
    )
    prompt = prompt_template.format(ins=ins, outp=outp, criteria=criteria)
    return sys_prompt, prompt
system_prompt, user_prompt = gen_prompt_no_input(ins=structured_sample["instruction"], outp=structured_sample["instances"][0]["output"])
response = llm.request(system_prompt=system_prompt, user_prompt=user_prompt, max_tokens=2048)
insert_to_db(collection=collection, data=response)
print(response)

👾 Warning: Failed to refine the output of LLM because: Extra data: line 1 column 2 (char 1)
🔥 Successfully Log Request to Database


{'request_id': 'chatcmpl-9sPcokZQUIkrR8bSS2pdj4YPQSzPK',
 'output': '1. This answer is not good for the given instruction because:\n- Helpfulness: The answer is not as helpful as it could be because it does not effectively simplify the sentence or provide a clearer alternative.\n- Relevance: The answer does not directly address the need to rewrite the sentence to make it clearer and concise.\n- Accuracy: The answer does not accurately assess the issues with the original sentence and does not provide a suitable revision.\n- Level of Details: The answer lacks specific details on how to improve the sentence, making it less informative.\n- Structure: The answer does not follow a structured approach to breaking down the original sentence and simplifying it effectively.\n\n2. [Better Answer] If you have any questions about my rate or need to adjust the scope of this project, please inform me promptly. [End]',
 'completion_tokens': 164,
 'prompt_tokens': 237,
 'total_tokens': 401}